In [69]:
import random
import requests


# Avaliador de jogadores de Forca

Começamos esse projeto com uma clase que simula um jogo de forca (com a funcao de advinhar a letra e/ou a palavra):


In [70]:
# Class imported from notebook 7 

# -------------- #
# List of words
url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
r = requests.get(url, allow_redirects=True)
content = str(r.content.decode()).split('\n')
# -------------- #

class Hangman:
    def __init__(self):
        
        if r.status_code==200:
            self.content = content
        else:
            print("Erro: ", r.status_code)
    
    # ----------------------------- #

    def new_game(self, lifes=5):
        self.running = True
        self.lifes = lifes
        self.word = random.choice(self.content)
        return len(self.word)
    
    # ----------------------------- #

    def guess_letter(self, letter):
        if self.lifes > 0:
            if letter in self.word:
                return [idx for idx in range(len(self.word)) if self.word[idx]==letter]
            else:
                self.lifes -= 1
                if self.lifes == 0:
                    self.running = False
                    return False
                else:
                    return []
    
    # ----------------------------- #

    def guess_word(self, word):
        if self.lifes > 0:
            if self.word == word:
                self.running = False
                return True
            else:
                self.lifes = 0
                self.running = False
                return False
            
    # ----------------------------- #

    def check_word(self, word):
        if self.word == word:
            return True
        return False

Para demonstrar o seu funcionamento antes de entrar nos algoritimos:

In [71]:
# Showcase

game = Hangman()
letters = game.new_game()
print("Letters:" + str(letters))

print(game.guess_letter('2'))
print(game.guess_letter('q'))
print(game.guess_letter('a'))
print(game.guess_letter('y'))
print(game.guess_letter('z'))
print(game.guess_letter('x'))

'Lifes: ', game.lifes


Letters:10
[]
[]
[3, 6, 8]
[]
[]
False


('Lifes: ', 0)

---------
## Algoritimo 1: Letra mais provável

O primeiro algoritimo feito simplismente ve qual a probabilidade de cada letra aparecer, considerando todas as palavras ao mesmo tempo e ignorando as suas posições.
Ele não leva em consideração o estado atual do tabuleiro, e sempre chuta letras na mesma ordem. 

In [75]:
# First algorithm: by absolute letter probability, wo word guesses

alphabet = 'abcdefghijklmnopqrstuvwxyz'
probabilities = {letter:0 for letter in alphabet}

# adds 1 for every time that letter appears
for word in content:
    for letter in word:
        probabilities[letter.lower()] += 1

# Orders the probabilities
probabilities = dict(sorted(probabilities.items(), key=lambda item: item[1], reverse=True))


Para testar sua eficácia, simulamos 1000 jogos com ele e vemos a chance de ele ganhar um jogo:

In [76]:
# 1000 runs, checks the win probability

wins = 0
keysProbs = list(probabilities.keys())
for i in range(1000):

    # starts game
    game = Hangman()
    letters = game.new_game()  

    # makes a list to save the guesses
    guessList = []
    for i in range(letters):
        guessList.append('_')

    curLetter = 0 
    while game.running:

        # guesses a letter and saves it 
        letterSpots = game.guess_letter(keysProbs[curLetter])
        if type(letterSpots) != bool:
            for numbs in letterSpots:
                guessList[numbs] = keysProbs[curLetter]
            curLetter+=1

        # checks if the word is already guessed
        guessWord = ''.join(guessList)
        gotWord = game.check_word(guessWord)
        if gotWord:
            break

    if gotWord:
        wins+=1
    # print(guessWord) # uncomment to see what the games ended on!

print('This algorithm wins '+ str(wins/10) +'%. of the time')

This algorithm wins 17.5%. of the time


Como podemos ver, se desconsiderar-mos o estado atual do jogo e sempre chutar em uma mesma ordem, não conseguimos ir muito longe. Ele acaba sendo melhor do que um chute completamente aleatorio, mas não por uma margem muito grande.

---------
## Algoritimo 2: 
